In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model


## Ler Dados

In [2]:
IdadePaciente = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="IdadePaciente")
DisponPaciente = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="DisponPaciente")
LocalPaciente = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="LocalPaciente")
RegraProfissional = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="RegraProfissional")
DisponProfissional = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="DisponProfissional")
LocalProfissional = pd.read_excel("C:/unisoma/docs/cenario_2.xlsx", sheet_name="LocalProfissional")

## Funcoes

In [3]:
def clean1(df, col1):
    df[col1] = df[col1].ffill()
    return df

def clean2(df, col1:str, col2:str, colname:str):
    df = df.fillna(0)
    df = df.map(lambda x: 1 if x in ["X", "x"] else x)
    if col2 != "":
        df = pd.melt(df, id_vars=[col1,col2], var_name=colname, value_name='valor')
    else:
        df = pd.melt(df, id_vars=[col1], var_name=colname, value_name='valor')

    df["valor"] = df["valor"].astype(int)
    return df

## Limpeza

In [4]:
DisponPaciente = clean1(DisponPaciente,'paciente')
DisponPaciente =  clean2(DisponPaciente,  'paciente', 'dia_semana', 'hora')

LocalPaciente = clean1(LocalPaciente,'paciente')
LocalPaciente = clean2(LocalPaciente, 'paciente', 'dia_semana', 'lugar')

DisponProfissional = clean1(DisponProfissional,'profissional')
DisponProfissional = clean2(DisponProfissional, 'profissional', 'dia_semana', 'hora')

RegraProfissional = clean1(RegraProfissional, "profissional")
RegraProfissionalHoras = RegraProfissional[["profissional","tipo","horas_semana"]]
RegraProfissionalGente = RegraProfissional[["profissional","infantil","adolescente","adulto"]]
RegraProfissionalGente = clean2(RegraProfissionalGente, 'profissional', "", 'tipoPaciente')

LocalProfissional = clean1(LocalProfissional, "profissional")
LocalProfissional = clean2(LocalProfissional, 'profissional', "", 'lugar')


In [5]:
DisponPaciente["paciente"].unique()

array(['paciente1', 'paciente2', 'paciente3', 'paciente4'], dtype=object)

## Conjuntos y Parametros

In [6]:
# CONJUNTOS
P = IdadePaciente["paciente"].unique().tolist()
E = RegraProfissionalHoras["profissional"].unique().tolist()
K = RegraProfissionalGente["tipoPaciente"].unique().tolist()
L = LocalProfissional["lugar"].unique().tolist()
H = DisponPaciente["hora"].unique().tolist()
D = DisponPaciente["dia_semana"].unique().tolist()

# PARAMETROS
DPH = DisponPaciente.set_index(['paciente', 'dia_semana', 'hora'])['valor'].to_dict()
DPL = LocalPaciente.set_index(['paciente', 'dia_semana', 'lugar'])['valor'].to_dict()
NHD = RegraProfissionalHoras.set_index(['profissional'])['horas_semana'].to_dict()
AP = RegraProfissionalGente.set_index(['profissional','tipoPaciente'])['valor'].to_dict()
DEH = DisponProfissional.set_index(['profissional', 'dia_semana', 'hora'])['valor'].to_dict()
DL = LocalProfissional.set_index(['profissional', 'lugar'])['valor'].to_dict()

## Modelo

In [17]:
# Crear el solver (CBC, para programación entera mixta)
# model = pywraplp.Solver.CreateSolver('GLOP')  # Usa 'GLOP' para programación lineal o 'SCIP' para programación entera
model = cp_model.CpModel()

#criar as variaveis de desicao
x = {}
for i in P:
    for j in E:
        for d in D:
            for h in H:
                for l in L:
                    x[(i, j, d, h, l)] = model.NewBoolVar(f'var_{i}_{j}_{d}_{h}_{l}')

# Criar a Funcao Objetivo
model.Maximize(sum(x[i,j,d,h,l] for i in P for j in E for d in D for h in H for l in L))


# CRIAR AS RESTRICOES

# Capacidade semanal de cada profissional
for j in E:
    model.Add(sum(x[(i,j,d,h,l)]  for i in P for d in D for h in H for l in L) <= NHD[j])


# Cada profissional deve atender no máximo um paciente em cada horario
for j in E:
    for d in D:
        for h in H:
            model.Add(sum(x[(i,j,d,h,l)]  for i in P for l in L) <= 1)


# Cada paciente so tem uma sessao semanal como máximo
for i in P:
    model.Add(sum(x[(i,j,d,h,l)]  for j in E for d in D for h in H for l in L) <= 1)


# Para cada Doutor, para cada dia, fara-se atendimentos em máximo um lugar
for j in E:
    for d in D:
        model.Add(sum(x[(i,j,d,h,l)]  for i in P for h in H for l in L if j != "virtual_epsi") <= 1)


# Cada medico atendera nas franjas horarias disponibles
for j in E:
    for d in D:
        for h in H:
            model.Add(sum(x[(i,j,d,h,l)]  for i in P for l in L) <= DEH[j,d,h])


# Cada paciente será atendido nas franjas horarias disponibles
for i in P:
    for d in D:
        for h in H:
            model.Add(sum(x[(i,j,d,h,l)]  for j in E for l in L) <= DPH[(i,d,h)])


for j in E:
    for l in L:
        model.Add(sum(x[(i,j,d,h,l)]  for i in P for d in D for h in H) <= DL[(j,l)])



# Criar o solucionador
solver = cp_model.CpSolver()

# Resolver o modelo
status = solver.Solve(model)

resultado = pd.DataFrame(columns=["Paciente","Profesional","Dia","Hora","Lugar","Valor"])

# Imprimir os resultados
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solucao:')
    for key, var in x.items():
        if solver.Value(var) == 1:
            newRegistro = pd.DataFrame({'Paciente': [key[0]], 'Profesional': [key[1]], 'Dia': [key[2]],'Hora': [key[3]], 'Lugar': [key[4]], 'Valor': [solver.Value(var)]})
            resultado = pd.concat([resultado, newRegistro], ignore_index=True)
            print(f'x_{key} Foi Atribuido')

    print("")
    print("FO: ",solver.objective_value)
else:
    print('Sem Solucao')

resultado.to_excel("resultado.xlsx", index=False)

Solucao:
x_('paciente1', 'profissional3', 'seg', 'hr_12', 'ASA - PÁSSAROS') Foi Atribuido
x_('paciente2', 'profissional1', 'sáb', 'hr_18', 'virtual_epsi') Foi Atribuido
x_('paciente3', 'profissional2', 'qui', 'hr_10', 'INSTITUTO BORBOLETA AZUL') Foi Atribuido
x_('paciente4', 'profissional4', 'sáb', 'hr_16', 'ASA - SANTA MÔNICA') Foi Atribuido

FO:  4.0


In [9]:

# Crear el solver (CBC, para programación entera mixta)
solver = pywraplp.Solver.CreateSolver('CBC')

# Definir el número de personas y trabajos
n = 3  # Puedes cambiarlo a más personas/trabajos

# Ejemplo de matriz de costos
costos = [
    [90, 76, 75],
    [35, 85, 55],
    [125, 95, 90]
]

# Definir variables binarias x[i][j]: si la persona i está asignada al trabajo j
x = []
for i in range(n):
    x.append([solver.BoolVar(f'x[{i},{j}]') for j in range(n)])

# # Restricción 1: Cada persona solo puede hacer un trabajo
# for i in range(n):
#     solver.Add(solver.Sum(x[i][j] for j in range(n)) == 1)

# # Restricción 2: Cada trabajo solo puede ser hecho por una persona
# for j in range(n):
#     solver.Add(solver.Sum(x[i][j] for i in range(n)) == 1)

# # Definir la función objetivo: minimizar el costo total
# solver.Minimize(solver.Sum(costos[i][j] * x[i][j] for i in range(n) for j in range(n)))

# # Resolver el problema
# status = solver.Solve()

# # Imprimir los resultados
# if status == pywraplp.Solver.OPTIMAL:
#     print('Solución óptima encontrada:')
#     for i in range(n):
#         for j in range(n):
#             if x[i][j].solution_value() == 1:
#                 print(f'Persona {i} asignada al trabajo {j} con un costo de {costos[i][j]}')
#     print(f'Costo total mínimo: {solver.Objective().Value()}')
# else:
#     print('No se encontró solución óptima.')
x

[[x[0,0], x[0,1], x[0,2]], [x[1,0], x[1,1], x[1,2]], [x[2,0], x[2,1], x[2,2]]]

In [10]:
from ortools.linear_solver import pywraplp

# Crear el solver (usaremos el método de simplex)
solver = pywraplp.Solver.CreateSolver('GLOP')

# Definir las variables x y y
x = solver.NumVar(0, solver.infinity(), 'x')
y = solver.NumVar(0, solver.infinity(), 'y')

# Definir las restricciones
solver.Add(x + 2 * y <= 14)
solver.Add(3 * x - y >= 0)
solver.Add(x - y <= 2)

# Definir la función objetivo (maximizar 3x + 4y)
solver.Maximize(3 * x + 4 * y)

# Resolver el problema
status = solver.Solve()

# Imprimir los resultados
if status == pywraplp.Solver.OPTIMAL:
    print('Solución:')
    print(f'x = {x.solution_value()}')
    print(f'y = {y.solution_value()}')
    print(f'Valor óptimo = {solver.Objective().Value()}')
else:
    print('El problema no tiene solución óptima.')


Solución:
x = 5.999999999999998
y = 3.9999999999999996
Valor óptimo = 33.99999999999999
